In [35]:
import pandas, numpy
from novelty_experiment_runner_cartpole import NoveltyExperimentRunnerCartpole
from os import path
import settings
import constants


In [36]:
results_file = open(path.join(settings.ROOT_PATH, "data", "cartpole", "test", "repairing_test_wsu_copy.csv"), "r")


In [37]:
dataframe = pandas.read_csv(results_file)

In [38]:
dataframe['is_novel'] = numpy.where(dataframe['novelty_probability'] < dataframe['novelty_threshold'], False, True)

In [39]:
dataframe['TN'] = numpy.where( (dataframe['type'] == constants.NON_NOVELTY_PERFORMANCE) & (dataframe['is_novel'] == False), 1, 0)
dataframe['FP'] = numpy.where((dataframe['type'] == constants.NON_NOVELTY_PERFORMANCE) & (dataframe['is_novel'] == True), 1, 0)
dataframe['TP'] = numpy.where((((dataframe['type'] == constants.UNKNOWN_NOVELTY) | (dataframe['type'] == constants.KNOWN_NOVELTY)) & (dataframe['is_novel'] == True)),
                                      1, 0)
dataframe['FN'] = numpy.where((((dataframe['type'] == constants.UNKNOWN_NOVELTY) | (dataframe['type'] == constants.KNOWN_NOVELTY)) & (dataframe['is_novel'] == False)),
                                      1, 0)


In [40]:
print(dataframe)

    Unnamed: 0  trial_num  episode_num                     type  \
0            0          0            0  non-novelty-performance   
1            0          0            1  non-novelty-performance   
2            0          0            2  non-novelty-performance   
3            0          0            3          unknown-novelty   
4            0          0            4          unknown-novelty   
5            0          0            5          unknown-novelty   
6            0          0            3            known-novelty   
7            0          0            4            known-novelty   
8            0          0            5            known-novelty   
9            0          1            0  non-novelty-performance   
10           0          1            1  non-novelty-performance   
11           0          1            2  non-novelty-performance   
12           0          1            3          unknown-novelty   
13           0          1            4          unknown-novelt

1. Create trial groups

In [48]:
trials = dataframe[['trial_num', 'FN', 'FP', 'TN', 'TP', 'performance']].groupby('trial_num').agg({'FN': numpy.sum, 'FP': numpy.sum, 'TN': numpy.sum, 'TP': numpy.sum, 'performance': numpy.mean})

In [51]:
trials['is_CDT'] = numpy.where((trials['TP'] > 1) & (trials['FP'] ==0), True, False)

In [52]:
print(trials)

           FN  FP  TN  TP  performance  is_CDT
trial_num                                     
0           6   0   3   0        200.0   False
1           6   0   3   0        200.0   False


In [53]:
CDT_trials = trials[trials['is_CDT'] == False]

In [54]:
print(CDT_trials)

           FN  FP  TN  TP  performance  is_CDT
trial_num                                     
0           6   0   3   0        200.0   False
1           6   0   3   0        200.0   False


In [55]:
M1 = CDT_trials['FN'].mean()
print("M1 is {}".format(M1))

M1 is 6.0


In [56]:
M2 = len(CDT_trials)/len(pandas.unique(dataframe['trial_num']))
print("M2 is {}".format(M2))

M2 is 1.0
